# Морской бой

```{note}
:class: dropdown 
**Правила**

Цель игры:
Цель игры — уничтожить все корабли противника первым.

Корабли:
Игроки расставляют свои корабли на своём поле.
- 1 корабль длиной 4 клетки 
- 2 корабля длиной 3 клетки 
- 3 корабля длиной 2 клетки 
- 4 корабля длиной 1 клетка.
Корабли могут размещаться горизонтально или вертикально и не должны касаться друг друга.

Процесс игры :
Игроки стреляют по очереди (указанием на клетку поля противника, например, 1 1.
Если выстрел попал в корабль противника, объявляется "попадание" (и эта клетка помечается). Если все клетки корабля были поражены, корабль считается потопленным.
Если выстрел пришелся на пустую клетку, объявляется "промах".

Победа:
Побеждает тот, кто первым потопит все корабли противника.

ВЫХОД ИЗ ИГРЫ: КЛАВИША "q".
```

## Импорт бибилотек

In [1]:
import cv2
import numpy as np
import random

## Визуал

In [2]:
# Размеры игрового поля
GRID_SIZE = 10
CELL_SIZE = 30
MARGIN = 5

# Цвета для отображения
BACKGROUND_COLOR = (255, 255, 255)
LINE_COLOR = (0, 0, 0)
SHIP_COLOR = (192, 192, 192)
HIT_SHIP_COLOR = (0, 0, 255)  # Красный для попаданий
MISS_COLOR = (0, 255, 255)  # Желтый для промахов

# Загрузка изображения корабля
ship_image = cv2.imread(r'ship.jpg')
ship_image = cv2.resize(ship_image, (CELL_SIZE, CELL_SIZE))

## Отрисовка

In [3]:
def create_board():
    return np.zeros((GRID_SIZE, GRID_SIZE), dtype=np.int32)

def can_place_ship(board, row, col, length, orientation):
    if orientation == 'horizontal':
        if col + length > GRID_SIZE:
            return False
        for i in range(max(0, col-1), min(GRID_SIZE, col+length+1)):
            for j in range(max(0, row-1), min(GRID_SIZE, row+2)):
                if board[j, i] == 1:
                    return False
    elif orientation == 'vertical':
        if row + length > GRID_SIZE:
            return False
        for i in range(max(0, row-1), min(GRID_SIZE, row+length+1)):
            for j in range(max(0, col-1), min(GRID_SIZE, col+2)):
                if board[i, j] == 1:
                    return False
    return True

## Клик и мастшаб

In [4]:
def mouse_callback(event, x, y, flags, param):
    global user_click, scale
    if event == cv2.EVENT_LBUTTONDOWN:
        img_width = GRID_SIZE * CELL_SIZE + MARGIN * (GRID_SIZE + 1)
        if y > img_width * scale:
            y -= int(img_width * scale)
            col = int((x - MARGIN * scale) / ((CELL_SIZE + MARGIN) * scale))
            row = int((y - MARGIN * scale) / ((CELL_SIZE + MARGIN) * scale))
            if 0 <= row < GRID_SIZE and 0 <= col < GRID_SIZE:
                user_click = (row, col)

user_click = None
scale = 1 # Масштабирование

## Логика

In [5]:
def place_ships(board, ships):
    for length in ships:
        placed = False
        while not placed:
            orientation = random.choice(['horizontal', 'vertical'])
            if orientation == 'horizontal':
                row = random.randint(0, GRID_SIZE - 1)
                col = random.randint(0, GRID_SIZE - length)
                if can_place_ship(board, row, col, length, 'horizontal'):
                    board[row, col:col + length] = 1
                    placed = True
            else:
                row = random.randint(0, GRID_SIZE - length)
                col = random.randint(0, GRID_SIZE - 1)
                if can_place_ship(board, row, col, length, 'vertical'):
                    board[row:row + length, col] = 1
                    placed = True

def draw_boards(player_board, player_shown, computer_shown, scale=1):
    img_width = GRID_SIZE * CELL_SIZE + MARGIN * (GRID_SIZE + 1)
    img_height = 2 * img_width + MARGIN

    img = np.ones((img_height, img_width, 3), dtype=np.uint8) * 255

    for board_idx, (board, shown_board) in enumerate([(player_board, player_shown), (None, computer_shown)]):
        offset_y = board_idx * (img_width + MARGIN)

        for row in range(GRID_SIZE):
            for col in range(GRID_SIZE):
                top_left = (col * CELL_SIZE + MARGIN * (col + 1),
                            row * CELL_SIZE + MARGIN * (row + 1) + offset_y)
                bottom_right = (top_left[0] + CELL_SIZE, top_left[1] + CELL_SIZE)

                if shown_board[row, col] == 2:
                    # Попадание по кораблю - показываем изображение или красный цвет
                    if board_idx == 1:  # компьютера
                        img[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]] = ship_image
                    else:  # игрока
                        cv2.rectangle(img, top_left, bottom_right, HIT_SHIP_COLOR, thickness=-1)
                elif shown_board[row, col] == 3:
                    # Промах
                    cv2.circle(img, ((top_left[0] + bottom_right[0]) // 2,
                                     (top_left[1] + bottom_right[1]) // 2),
                               CELL_SIZE // 4, MISS_COLOR, thickness=2)

                if board_idx == 0 and board is not None and board[row, col] == 1 and shown_board[row, col] != 2:
                    img[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]] = ship_image

                cv2.rectangle(img, top_left, bottom_right, LINE_COLOR, thickness=2)

    new_img_width = int(img_width * scale)
    new_img_height = int(img_height * scale)
    img = cv2.resize(img, (new_img_width, new_img_height))

    return img

def shoot(board, shown_board, row, col):
    if board[row, col] == 1:
        shown_board[row, col] = 2
        return True
    else:
        shown_board[row, col] = 3
        return False

def get_neighbors(row, col):
    return [(row-1, col), (row+1, col), (row, col-1), (row, col+1)]

## Запуск

In [7]:
def main():
    global user_click, scale

    player_board = create_board()
    computer_board = create_board()

    ships = [4, 3, 3, 2, 2, 2, 1, 1, 1, 1]
    place_ships(player_board, ships)
    place_ships(computer_board, ships)

    shown_player_board = np.zeros_like(player_board)
    shown_computer_board = np.zeros_like(computer_board)

    hunt_targets = []  # Цели компьютера для уничтожения
    game_over = False

    cv2.namedWindow('Морской бой', cv2.WINDOW_NORMAL)
    cv2.setMouseCallback('Морской бой', mouse_callback)

    while not game_over:
        board_img = draw_boards(player_board, shown_player_board, shown_computer_board, scale=scale)
        cv2.imshow('Морской бой', board_img)
        key = cv2.waitKey(10) & 0xFF
        if key == ord('q'):
            break

        if user_click is not None:
            row, col = user_click
            user_click = None

            if shown_computer_board[row, col] != 0:
                print("Сюда уже стреляли! Попробуйте другую точку.")
                continue

            hit = shoot(computer_board, shown_computer_board, row, col)
            if hit:
                print("Попадание!")
            else:
                print("Промах!")

            if np.all((shown_computer_board == 2) == (computer_board == 1)):
                game_over = True
                print("Вы победили!")
                break

            # Умная стрельба компьютера
            if hunt_targets:
                comp_row, comp_col = hunt_targets.pop()
            else:
                comp_row, comp_col = random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1)
                while shown_player_board[comp_row, comp_col] != 0:
                    comp_row, comp_col = random.randint(0, GRID_SIZE - 1), random.randint(0, GRID_SIZE - 1)

            shot_result = shoot(player_board, shown_player_board, comp_row, comp_col)
            if shot_result:
                print(f"Компьютер попал: {comp_row}, {comp_col}")
                # Добавляем соседние клетки в очередь для дальнейшего поиска и уничтожения
                hunt_targets.extend(
                    [(r, c) for r, c in get_neighbors(comp_row, comp_col)
                     if 0 <= r < GRID_SIZE and 0 <= c < GRID_SIZE and shown_player_board[r, c] == 0]
                )
            else:
                print(f"Компьютер промахнулся: {comp_row}, {comp_col}")

            if np.all((shown_player_board == 2) == (player_board == 1)):
                game_over = True
                print("Компьютер победил!")
                break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Промах!
Компьютер промахнулся: 7, 3
Промах!
Компьютер промахнулся: 1, 4
Промах!
Компьютер промахнулся: 2, 3
Промах!
Компьютер попал: 2, 2
Промах!
Компьютер промахнулся: 2, 1
Сюда уже стреляли! Попробуйте другую точку.
Промах!
Компьютер промахнулся: 3, 2
Попадание!
Компьютер промахнулся: 1, 2
Промах!
Компьютер промахнулся: 0, 6
